In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
folder = 'sol/'

# Get the files in the folder
import os
files = os.listdir(folder)
# Match files that end in vr.dat
files_vr = [f for f in files if f.endswith('vr.dat')]
files_vr.sort()
files_sol = [f for f in files if f.endswith('solution.dat')]
files_sol.sort()

In [ ]:
ALL_XS = []
ALL_YS = []
ALL_ZS = []
ALL_UXS = []
ALL_UYS = []
ALL_UZS = []
ALL_QXS = []
ALL_QYS = []
ALL_QZS = []


for f in files_vr:
    XS = []
    YS = []
    ZS = []
    UXS = []
    UYS = []
    UZS = []
    QXS = []
    QYS = []
    QZS = []
    with open(folder + f) as file:
        lines = file.readlines()
        vars = lines[0].split()
        
        # From the file name get the NTIME
        ntime = int(f[:5])

        for l in lines[1:]:
            l = l.split()
            id = int(l[0])
            x = float(l[1])
            y = float(l[2])
            z = float(l[3])
            u = float(l[4])
            v = float(l[5])
            w = float(l[6])
            vort_x = float(l[7])
            vort_y = float(l[8])
            vort_z = float(l[9])

            XS.append(x)
            YS.append(y)
            ZS.append(z)
            UXS.append(u)
            UYS.append(v)
            UZS.append(w)
            QXS.append(vort_x)
            QYS.append(vort_y)
            QZS.append(vort_z)

    XS = np.array(XS)
    YS = np.array(YS)
    ZS = np.array(ZS)
    UXS = np.array(UXS)
    UYS = np.array(UYS)
    UZS = np.array(UZS)
    QXS = np.array(QXS)
    QYS = np.array(QYS)
    QZS = np.array(QZS)

    ALL_XS.append(XS)
    ALL_YS.append(YS)
    ALL_ZS.append(ZS)
    ALL_UXS.append(UXS)
    ALL_UYS.append(UYS)
    ALL_UZS.append(UZS)
    ALL_QXS.append(QXS)
    ALL_QYS.append(QYS)
    ALL_QZS.append(QZS)

In [ ]:
ALL_PM_XS = []
ALL_PM_YS = []
ALL_PM_ZS = []
ALL_PM_UXS = []
ALL_PM_UYS = []
ALL_PM_UZS = []
ALL_PM_QXS = []
ALL_PM_QYS = []
ALL_PM_QZS = []


for f in files_sol:
    XS = []
    YS = []
    ZS = []
    UXS = []
    UYS = []
    UZS = []
    QXS = []
    QYS = []
    QZS = []
    with open(folder + f) as file:
        lines = file.readlines()
        vars = lines[0].split()
        
        # From the file name get the NTIME
        ntime = int(f[:5])
        sizes = [i for i in lines[1].split()]
        IS = int(sizes[1][2:])
        JS = int(sizes[2][2:])
        KS = int(sizes[3][2:])

        for l in lines[3:]:
            l = l.split()
            x = float(l[0])
            y = float(l[1])
            z = float(l[2])
            u = float(l[3])
            v = float(l[4])
            w = float(l[5])
            vort_x = float(l[6])
            vort_y = float(l[7])
            vort_z = float(l[8])

            XS.append(x)
            YS.append(y)
            ZS.append(z)
            UXS.append(u)
            UYS.append(v)
            UZS.append(w)
            QXS.append(vort_x)
            QYS.append(vort_y)
            QZS.append(vort_z)

    # Convert from Fortran to C
    XS = np.array(XS).reshape(KS, JS, IS, order='F').T
    YS = np.array(YS).reshape(KS, JS, IS, order='F').T
    ZS = np.array(ZS).reshape(KS, JS, IS, order='F').T
    UXS = np.array(UXS).reshape(KS, JS, IS, order='F').T
    UYS = np.array(UYS).reshape(KS, JS, IS, order='F').T
    UZS = np.array(UZS).reshape(KS, JS, IS, order='F').T
    QXS = np.array(QXS).reshape(KS, JS, IS, order='F').T
    QYS = np.array(QYS).reshape(KS, JS, IS, order='F').T
    QZS = np.array(QZS).reshape(KS, JS, IS, order='F').T


    ALL_PM_XS.append(XS)
    ALL_PM_YS.append(YS)
    ALL_PM_ZS.append(ZS)
    ALL_PM_UXS.append(UXS)
    ALL_PM_UYS.append(UYS)
    ALL_PM_UZS.append(UZS)
    ALL_PM_QXS.append(QXS)
    ALL_PM_QYS.append(QYS)
    ALL_PM_QZS.append(QZS)

In [ ]:
print(len(ALL_XS))
print(len(ALL_PM_XS))
print(ALL_PM_XS[0].shape)
x_min = np.min(ALL_PM_XS[0])
x_max = np.max(ALL_PM_XS[0])
y_min = np.min(ALL_PM_YS[0])
y_max = np.max(ALL_PM_YS[0])
print(x_min, x_max)
print(y_min, y_max)

In [ ]:
ALL_PM_XS[0].shape

In [ ]:
ALL_PM_XS[0][:,:,0]

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim(x_min, x_max)
ax.set_xlim(y_min, y_max)
ax.set_aspect('equal')
time = 40
V_MAG = np.sqrt(ALL_PM_UXS[time][:,:,0]**2 + ALL_PM_UYS[time][:,:,0]**2 + ALL_PM_UZS[time][:,:,0]**2)


# Color the particle mesh by the velocity magnitude
c = V_MAG
# Contour the velocity magnitude
# c = ax.contour(ALL_PM_XS[time][:,:,0], ALL_PM_YS[time][:,:,0], V_MAG, cmap='viridis')
# Scatter plot XS and YS
sc = ax.scatter(ALL_PM_XS[time][:,:,0], ALL_PM_YS[time][:,:,0], c=c, cmap='viridis')

In [ ]:
# Make a movie of the paricles moving in X-Y plane
from matplotlib.animation import FuncAnimation
fig, ax = plt.subplots()
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect('equal')
scat = ax.scatter([], [], s=30)

# Color the particle mesh by the velocity magnitude
# con = ax.contour(ALL_PM_XS[0][:,:,0], ALL_PM_YS[0][:,:,0], np.sqrt(ALL_PM_UXS[0][:,:,0]**2 + ALL_PM_UYS[0][:,:,0]**2), cmap='viridis')
sc = ax.scatter(ALL_PM_XS[time][:,:,0], ALL_PM_YS[time][:,:,0], c=c, cmap='viridis', s = 5)

def update(frame):
    scat.set_offsets(np.column_stack((ALL_XS[frame], ALL_YS[frame])))
    V_MAG = np.sqrt(ALL_PM_UXS[frame]**2 + ALL_PM_UYS[frame]**2 + ALL_PM_UZS[frame]**2)
    V_MAG = V_MAG[:,:,0]
    print(frame)
    print(V_MAG.shape)
    print(np.min(V_MAG), np.max(V_MAG)) 
    # con = ax.contour(ALL_PM_XS[frame][:,:,0], ALL_PM_YS[frame][:,:,0], V_MAG, cmap='viridis')
    sc = ax.scatter(ALL_PM_XS[time][:,:,0], ALL_PM_YS[time][:,:,0], c=c, cmap='viridis')
    # Write the frame number
    ax.set_title(f'Frame {frame}')
    print(ALL_XS[frame].shape, len(ALL_XS), frame)
    return scat,

ani = FuncAnimation(fig, update, frames=60, blit=True)
ani.save('movie.mp4')